In [1]:
# import spark sql
from pyspark.sql import SparkSession
# create spark session with 16 threads
spark = SparkSession.builder.master('local[16]').getOrCreate()

In [2]:
# import spark sql functions
from pyspark.sql.functions import *

In [3]:
# load student_data.csv
student_data = spark.read.csv('student_data.csv', header=True, inferSchema=True)

student_data.show()

+------+-----------+------------+------------+-------+---------+-----+--------+--------+-----------+----------------+-----------------+----------+---------------------+------------------------+------------------+-----+
|gender|NationalITy|PlaceofBirth|     StageID|GradeID|SectionID|Topic|Semester|Relation|raisedhands|VisITedResources|AnnouncementsView|Discussion|ParentAnsweringSurvey|ParentschoolSatisfaction|StudentAbsenceDays|Class|
+------+-----------+------------+------------+-------+---------+-----+--------+--------+-----------+----------------+-----------------+----------+---------------------+------------------------+------------------+-----+
|     M|         KW|      KuwaIT|  lowerlevel|   G-04|        A|   IT|       F|  Father|         15|              16|                2|        20|                  Yes|                    Good|           Under-7|    M|
|     M|         KW|      KuwaIT|  lowerlevel|   G-04|        A|   IT|       F|  Father|         20|              20|       

In [4]:
# student data schema
student_data.printSchema()

root
 |-- gender: string (nullable = true)
 |-- NationalITy: string (nullable = true)
 |-- PlaceofBirth: string (nullable = true)
 |-- StageID: string (nullable = true)
 |-- GradeID: string (nullable = true)
 |-- SectionID: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- Semester: string (nullable = true)
 |-- Relation: string (nullable = true)
 |-- raisedhands: integer (nullable = true)
 |-- VisITedResources: integer (nullable = true)
 |-- AnnouncementsView: integer (nullable = true)
 |-- Discussion: integer (nullable = true)
 |-- ParentAnsweringSurvey: string (nullable = true)
 |-- ParentschoolSatisfaction: string (nullable = true)
 |-- StudentAbsenceDays: string (nullable = true)
 |-- Class: string (nullable = true)



In [11]:
# get all unique values of StageID
student_data.select('StageID').distinct().show()

+------------+
|     StageID|
+------------+
|  HighSchool|
|MiddleSchool|
|  lowerlevel|
+------------+



In [5]:
# lets see topic frequency
student_data.groupBy('topic').count().show()

+---------+-----+
|    topic|count|
+---------+-----+
|  Science|   51|
|  Geology|   24|
|    Quran|   22|
|     Math|   21|
|Chemistry|   24|
|  English|   45|
|  Spanish|   25|
|  History|   19|
|       IT|   95|
|   French|   65|
|   Arabic|   59|
|  Biology|   30|
+---------+-----+



In [12]:
# lets see the topic that has the highest frequence where StageID == 'lowerlevel'
student_data.filter(student_data['StageID'] == 'lowerlevel').groupBy('topic').count().orderBy('count', ascending=False).show()

+-------+-----+
|  topic|count|
+-------+-----+
| French|   64|
|     IT|   62|
| Arabic|   31|
|Science|   27|
|History|    7|
|English|    6|
|   Math|    2|
+-------+-----+



In [13]:
student_data.filter(student_data['StageID'] == 'MiddleSchool').groupBy('topic').count().orderBy('count', ascending=False).show()

+---------+-----+
|    topic|count|
+---------+-----+
|  Biology|   30|
|   Arabic|   28|
|  English|   28|
|  Geology|   24|
|  Spanish|   24|
|Chemistry|   24|
|       IT|   23|
|  Science|   20|
|    Quran|   18|
|     Math|   17|
|  History|   12|
+---------+-----+



In [14]:
student_data.filter(student_data['StageID'] == 'HighSchool').groupBy('topic').count().orderBy('count', ascending=False).show()

+-------+-----+
|  topic|count|
+-------+-----+
|English|   11|
|     IT|   10|
|Science|    4|
|  Quran|    4|
|   Math|    2|
|Spanish|    1|
| French|    1|
+-------+-----+



In [15]:
# Show values from GradeID column
student_data.select('GradeID').distinct().show()

+-------+
|GradeID|
+-------+
|   G-06|
|   G-08|
|   G-04|
|   G-12|
|   G-09|
|   G-10|
|   G-02|
|   G-11|
|   G-05|
|   G-07|
+-------+



In [16]:
# Make a new Column called Grade, take string values from GradeID column and convert them to integer, first parse all non-integer values out
student_data = student_data.withColumn('Grade', regexp_replace('GradeID', '[^0-9]', '').cast('int'))

In [17]:
# Show values from Grade column
student_data.select('Grade').distinct().show()

+-----+
|Grade|
+-----+
|   12|
|    6|
|    5|
|    9|
|    4|
|    8|
|    7|
|   10|
|   11|
|    2|
+-----+



In [21]:
# Group by Nationality, StageID, Gender, compute the average of Grade column
student_data.groupBy('Nationality', 'StageID', 'Gender').agg(avg('Grade').alias('Grade')).orderBy('Grade', ascending=False).show()

+-----------+------------+------+------------------+
|Nationality|     StageID|Gender|             Grade|
+-----------+------------+------+------------------+
|        USA|  HighSchool|     F|              12.0|
|         KW|  HighSchool|     F|11.181818181818182|
|      Tunis|  HighSchool|     F|              11.0|
|      Tunis|  HighSchool|     M|              11.0|
|SaudiArabia|  HighSchool|     M|              11.0|
|SaudiArabia|  HighSchool|     F|              11.0|
|         KW|  HighSchool|     M|              10.6|
|   venzuela|  HighSchool|     M|              10.0|
|       Iran|  HighSchool|     M|               9.0|
|        USA|MiddleSchool|     M|               8.0|
|    lebanon|MiddleSchool|     M|               8.0|
|    Morocco|MiddleSchool|     M|               8.0|
|       Iran|MiddleSchool|     M|               8.0|
|      Syria|MiddleSchool|     M|               7.5|
|SaudiArabia|MiddleSchool|     F|               7.5|
|         KW|MiddleSchool|     M|            7

In [22]:
# Check correlation between VisitedResources and raisedhands
student_data.select(corr('VisitedResources', 'raisedhands')).show()

+-----------------------------------+
|corr(VisitedResources, raisedhands)|
+-----------------------------------+
|                 0.6915717054692968|
+-----------------------------------+



In [23]:
# Create a K-Mean (K=4) cluster with Raisedhands, VisitedResources, AnnouncementView and Discussion.
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
# Evaluators
from pyspark.ml.evaluation import ClusteringEvaluator
# Pipeline
from pyspark.ml import Pipeline


In [26]:
# schema
student_data.printSchema()

root
 |-- gender: string (nullable = true)
 |-- NationalITy: string (nullable = true)
 |-- PlaceofBirth: string (nullable = true)
 |-- StageID: string (nullable = true)
 |-- GradeID: string (nullable = true)
 |-- SectionID: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- Semester: string (nullable = true)
 |-- Relation: string (nullable = true)
 |-- raisedhands: integer (nullable = true)
 |-- VisITedResources: integer (nullable = true)
 |-- AnnouncementsView: integer (nullable = true)
 |-- Discussion: integer (nullable = true)
 |-- ParentAnsweringSurvey: string (nullable = true)
 |-- ParentschoolSatisfaction: string (nullable = true)
 |-- StudentAbsenceDays: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Grade: integer (nullable = true)



In [27]:
# Create a VectorAssembler object
assembler = VectorAssembler(inputCols=['raisedhands', 'VisITedResources', 'AnnouncementsView', 'Discussion'], outputCol='features')

In [28]:
# Transform the data
final_data = assembler.transform(student_data)

In [29]:
# Create a KMeans Model
kmeans = KMeans(featuresCol='features', k=4)

In [30]:
# Fit the model
model = kmeans.fit(final_data)

In [31]:
# Evaluate clustering by computing Silhouette score
evaluator = ClusteringEvaluator()

# Compute the silhouette score
silhouette = evaluator.evaluate(model.transform(final_data))

In [32]:
# Print the silhouette score
print("Silhouette with squared euclidean distance = " + str(silhouette))

Silhouette with squared euclidean distance = 0.5025542205209951


In [35]:
# Shows the result along with the cluster centers
centers = model.clusterCenters()
print("Cluster Centers: ")
for index, center in enumerate(centers):
    print("Cluster " + str(index) + ": " + str(center))

Cluster Centers: 
Cluster 0: [18.18023256 14.88953488 16.84883721 33.95348837]
Cluster 1: [27.4939759  73.78313253 32.07228916 37.61445783]
Cluster 2: [77.525      79.95833333 68.2        74.525     ]
Cluster 3: [73.71428571 76.40952381 42.44761905 27.34285714]


In [34]:
# Compute the mean of RaisedHands, VisitedResources, AnnouncementView and RaisedHands of each cluster and print.
model.transform(final_data).groupBy('prediction').agg(avg('raisedhands').alias('raisedhands'), avg('VisITedResources').alias('VisITedResources'), avg('AnnouncementsView').alias('AnnouncementsView'), avg('Discussion').alias('Discussion')).show()

+----------+------------------+-----------------+------------------+-----------------+
|prediction|       raisedhands| VisITedResources| AnnouncementsView|       Discussion|
+----------+------------------+-----------------+------------------+-----------------+
|         1| 27.49397590361446|73.78313253012048|  32.0722891566265| 37.6144578313253|
|         3| 73.71428571428571| 76.4095238095238| 42.44761904761905|27.34285714285714|
|         2|            77.525|79.95833333333333|              68.2|           74.525|
|         0|18.180232558139537|14.88953488372093|16.848837209302324|33.95348837209303|
+----------+------------------+-----------------+------------------+-----------------+



In [37]:
# Export as html with nbconvert
!jupyter nbconvert --to html student.ipynb

[NbConvertApp] Converting notebook student.ipynb to html
[NbConvertApp] Writing 617874 bytes to student.html
